## Introduction

This notebook is used to showcase brainmap plots using encoding results.


In [ ]:
import argparse
import glob
import os

import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pandas as pd

from tfsplt_encoding import organize_data
from tfsplt_brainmap import get_sigelecs, aggregate_data, Colorbar, read_coor, make_brainmap, plot_brainmap
from tfsplt_brainmap_cat import get_sigelec_df, make_brainmap_cat, plot_brainmap_cat

## Color Split Examples

Color split is a list consisting of floats and `Colorbar` objects. It is necessary for a brainmap plot.

Thinking of the color split as a horizontal number line. In the color split, if there is a float on the left of a `Colorbar`, it serves as the lower limit or minimum of the colorbar. Similarly, if there is a float on the right, it serves as the upper limit or maximum of the colorbar.

Inside the script, we will split the electrodes into different colorbars based on their effect values.

In [ ]:
# Color Split with one colorbar for all values
color_split = [Colorbar()]

# Color Split with one colorbar (only take values from 0.2 to 0.3)
color_split = [0.2, Colorbar(), 0.3]

# Color Split with two colorbars for all positive and negative values
color_split = [Colorbar(), 0, Colorbar()]

# Color Split with two colobars for really positive and negative values
# Any electrodes with effect values between -0.1 to 0.1 will not be plotted
color_split = [Colorbar(), -0.1, 0.1, Colorbar()]

`Colorbar` is a class defined in `tfsplt_brainmap.py`. Each is used to plot a colorbar in the brainmap.

Each `Colorbar` has these arguments:
- `title`: title of the colorbar, defaults to "correlation"
- `colorscale`: a colorscale of the colorbar, can be defined as a list of lists of colors, or a named colorscale defined here (https://plotly.com/python/builtin-colorscales/), defaults to red to yellow
  - red to yellow: `[[0, "rgb(255,0,0)"], [1, "rgb(255,255,0)"]]`
  - blue to white to red: `[[0, "rgb(0,0,255)"], [0.5, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]`
  - viridis colorscale: `"viridis"`
- `bar_min`: the minimum value for the colorscale, defaults to "min", which is the effect minimum. If a float is provided, any effect smaller than the float is adjusted to the float
- `bar_max`: the maximum of the colorscale, defaults to "max", which is the effect maximum. If a float is provided, any effect bigger than the float is adjusted to the float

In [ ]:
# For each colorbar object, if we don't provide any arguments, these are the defaults:
cbar = Colorbar()
print(cbar)

# Colorbar with customized title and colorscale
cbar = Colorbar(title="max-cor",colorscale="viridis")
print(cbar)

# Colorbar with customized range
# Electrode values smaller than 0.1 will be plotted as 0.1
# Electrode values bigger than 0.3 will be plotted as 0.3
cbar = Colorbar(bar_min = 0.1, bar_max = 0.3)
print(cbar)


## Brain Map Functions

We have two types of functions to plot brainmaps. One is `make_brainmap` and `make_brainmap_cat`, which plots and saves the brainmap given a dataframe with "electrode" and "effect" columns. The other is `plot_brainmap` and `plot_brainmap_cat`, which is the lower level function. It plots the brainmap given a dataframe with "electrode", "effect", and electrode coordinate ("MNI_X", "MNI_Y", "MNI_Z" for average, "T1_X", "T1_Y", "T1_Z" for individual) columns.

The `make_brainmap_cat` and `plot_brainmap_cat` functions plots electrode categories and creates a legend instead of a colobar. The most common use is plot electrodes colored by different subjects.

For both functions, we also need to provide some necessary args:
- args (namespace): commandline arguments
- df (pandas DataFrame): dataframe with electrodes and effects
- outfile (str): outfile name
  - Provide an empty string for `outfile` if you don't want the function to write to png

Note: Both functions return the 3D plotly object if needed

## Example 1 (247 Max)

Here we are plotting the maximum correlation across lags for each electrode based on the glove encoding results. This effectively is the same as [this](https://github.com/hassonlab/247-plotting/wiki/Brainmap-Max), except we will customize the colorbar

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625, 676, 7170, 798] # subjects
  formats = [ # encoding folder
    "/projects/HASSON/247/ken-encoding/demo/stock-glove/kw-tfs-full-%s-glove50-lag2k-25-all/*/*%s.csv"
  ]
  keys = ["comp","prod"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/20230510-tfs-sig-file/tfs-sig-file-glove-%(sid)s-%(key)s.csv"]
  lags_plot = np.arange(-2000,2001,25) # encoding lags
  lags_show = np.arange(-2000,2001,25) # lags for the effect
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../glove_%s.png"

# Aggregate Data
args = Args()
args = get_sigelecs(args) # get significant electrodes
df = aggregate_data(args) # aggregate data
df = organize_data(args, df) # trim data if necessary

# Get Effect
df["effect"] = df.max(axis=1) # get max per electrode
df.reset_index(inplace=True)

In [ ]:
# Customize Your Color Split Here
args.color_split = [Colorbar(title="max-cor",colorscale="viridis")]

# Brain Map Plots
for key in args.keys: # comp/prod
    df_plot = df.loc[df.key == key, ("electrode", "effect")]
    fig = make_brainmap(args, df_plot, args.outfile % key) # plot png

## Example 2 (247 Max Difference)

Here we are plotting the difference of maximum correlation across lags for each electrode based on the whisper-medium encoding results. This effectively is the same as [this](https://github.com/hassonlab/247-plotting/wiki/Brainmap-Max-Difference), except we will customize the colorbar range

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625, 676, 7170, 798] # subjects
  formats = [ # encoding folder
    "/projects/HASSON/247/ken-encoding/demo/20230520-whisper-medium/kw-tfs-full-%s-whisper-medium.en-encoder-lag5k-25-all-24/*/*%s.csv",
    "/projects/HASSON/247/ken-encoding/demo/20230520-whisper-medium/kw-tfs-full-%s-whisper-medium.en-decoder-lag5k-25-all-18/*/*%s.csv"
  ]
  keys = ["comp","prod"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-ende-outer-%(key)s.csv"]
  lags_plot = np.arange(-5000,5001,25) # encoding lags
  lags_show = np.arange(-5000,5001,25) # lags for the effect
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../whisper-diff_%s.png"


def get_part_df(label):  # get partial df
    idx = pd.IndexSlice
    part_df = df.loc[idx[label, :, :, :], :].copy()
    part_df.index = part_df.index.droplevel("label")
    part_df_idx = part_df.index.get_level_values("electrode").tolist()
    return part_df, part_df_idx

# Aggregate Data
args = Args()
args = get_sigelecs(args) # get significant electrodes
df = aggregate_data(args) # aggregate data
df = organize_data(args, df) # trim data if necessary

# Get Effect
df["max"] = df.max(axis=1) # get max per electrode
df1, _ = get_part_df("enc1") # get first encoding
df2, _ = get_part_df("enc2") # get second encoding
df1.loc[:, "effect"] = df1["max"] - df2["max"] # get difference
df = df1
df.reset_index(inplace=True)

In [ ]:
# Customize Your Color Split Here
pos_bar = Colorbar(title="Δ corr pos",colorscale=[[0, "rgb(255,248,240)"], [1, "rgb(255,0,0)"]],bar_max=0.15)
neg_bar = Colorbar(title="Δ corr neg",colorscale=[[0, "rgb(0,0,255)"], [1, "rgb(240,248,255)"]],bar_min=-0.15)
args.color_split = [neg_bar,-0.01,0.01,pos_bar]

# Brain Map Plots
for key in args.keys: # comp/prod
    df_plot = df.loc[df.key == key, ("electrode", "effect")]
    fig = make_brainmap(args, df_plot, args.outfile % key) # plot png

## Example 3 (247 Subjects)

Here we are plotting the 247 significant glove electrodes colored by subjects. This effectively is the same as [this](https://github.com/hassonlab/247-plotting/wiki/Brainmap-247-Subjects), except we use the customized colors here.

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625, 676, 7170, 798] # subjects
  keys = ["comp","prod"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/20230510-tfs-sig-file/tfs-sig-file-glove-%(sid)s-%(key)s.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../tfs_%s.png"

# Aggregate Data & Get Effect
args = Args()
df = get_sigelec_df(args)
df["effect"] = df.subject.astype(int)
df.loc[df.effect == 7170, "effect"] = 717  # fix for 717

In [ ]:
# Customize Your Color list Here
# Get qualitative color list (https://plotly.com/python/discrete-color/)
# color_list = px.colors.qualitative.Light24 # 24 colors
# color_list = px.colors.qualitative.Plotly # 10 colors
prop_cycle = plt.rcParams["axes.prop_cycle"] # get the encoding default colors
color_list = prop_cycle.by_key()["color"]

# Set Up Color Split
args.colors = color_list

# Brain Map Plots
for key in args.keys:
    df_plot = df.loc[df.key == key, ("electrode", "effect","subject")]
    fig = make_brainmap_cat(args, df_plot, args.outfile % key) # plot png

## Example 4 (Podcast Subjects)

Here we are plotting the significant podcast electrodes (160) colored by subjects. This effectively is the same as [this](https://github.com/hassonlab/247-plotting/wiki/Brainmap-Podcast-Subjects), except we use the customized colors here.

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "podcast"
  sid = [777] # subjects
  keys = ["comp"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/podcast_160.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../podcast.png"

# Aggregate Data & Get Effect
args = Args()
df = get_sigelec_df(args)
df["effect"] = df.subject.astype(int)

In [ ]:
# Customize Your Color list Here
color_list = px.colors.qualitative.Prism # 10 colors

# Set Up Color Split
args.colors = color_list

# Brain Map Plots
fig = make_brainmap_cat(args, df, args.outfile) # plot png

## Example 5 (247 Subjects)

Here we are plotting the significant whisper electrodes colored by type. We will call the lower level function `plot_brainmap_cat` here.

In [ ]:
class Args(argparse.Namespace):
  main_dir = "../data/plotting/brainplot/" # loads coordinate and brain surface files
  project = "tfs"
  sid = [625, 676, 7170, 798] # subjects
  keys = ["comp","prod"] # comprehension and/or production
  sig_elec_file = ["../data/plotting/sig-elecs/20230413-whisper-paper/tfs-sig-file-%(sid)s-whisper-ende-outer-%(key)s.csv"]
  brain_type = "ave" # average brain
  hemisphere = "left" # only plot left hemisphere
  outfile = "../tfs_%s.png"

# Aggregate Data
args = Args()
df = get_sigelec_df(args)
df.loc[df.subject == 7170, "subject"] = 717

# Merge with Electrode Coordinate File
subjects = df.subject.unique()
df_coor = read_coor(args.main_dir, subjects)
df = pd.merge(
    df.loc[:, ("subject", "electrode", "key")],
    df_coor,
    how="left",
    left_on=["subject", "electrode"],
    right_on=["subject", "name"],
)
df["effect"] = df.type # set type as effect

In [ ]:
# Customize Your Color list Here
color_list = px.colors.qualitative.D3 # 10 colors

# Set Up Color Split
args.colors = color_list

# Brain Map Plots
for key in args.keys:
    df_plot = df.loc[df.key == key, :]
    fig = plot_brainmap_cat(args, df_plot, args.outfile % key) # plot png